In [1]:
import pandas as pd
import numpy as np
from time import sleep
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException 
import re
import os

In [2]:


url="https://www.oddsportal.com/matches/tennis/"



driver = webdriver.Chrome(executable_path=r'/Users/pierremecchia/Desktop/chromedriver')
driver.get(url)
sleep(5)


table=driver.find_elements_by_xpath("//*[@id='table-matches']/table/tbody/tr[contains(@class,'dark')]")

list_href=[]
list_location=[]
list_surface=[]
list_player1=[]
list_player2=[]
list_odd1=[]
list_odd2=[]
list_winnerP1=[]


for row in table:
    if ("ATP" in row.text) and (not "Doubles" in row.text):
        href=row.find_element_by_xpath('./th[1]/a[2]').get_attribute('href')
        if href not in list_href:
            list_href.append(href)
driver.quit()


for href in list_href:
    
    driver = webdriver.Chrome(executable_path=r'/Users/pierremecchia/Desktop/chromedriver')
    driver.get(href)
    sleep(10)
    
    h1=driver.find_element_by_xpath('//*[@id="col-content"]/h1').text
    location=h1.replace('Betting Odds','')
    location=location.replace('ATP ','')
    location=re.sub(r'\([^)]*\)', '', location)
    location=location.rstrip().lstrip()
    matches=driver.find_elements_by_xpath("//*[@id='tournamentTable']/tbody/tr")
    
    eventNb=0
    for match in matches:
        #print(match.text)
        try:
            className=match.get_attribute("class")
            if className=='center nob-border':
                eventNb+=1
            if eventNb==1:
                if not('dark' or 'nob-border' or 'table-dummyrow')in className:
           
                    try:
                        test=match.find_element_by_xpath('./td[2]/a').text
                        if 'deactivate' in className:
                            players=match.find_element_by_xpath('./td[2]/a').text
                            winnerP1=match.find_element_by_xpath('./td[3]').text
                            odd1=match.find_element_by_xpath('./td[4]/a').text
                            odd2=match.find_element_by_xpath('./td[5]/a').text
                            if ("ret" in winnerP1) or ("canc" in winnerP1):
                                winnerP1=np.nan
                            elif int(winnerP1[0])<int(winnerP1[2]):
                                winnerP1=0
                            else:
                                winnerP1=1
                        else:
                            players=match.find_element_by_xpath('./td[2]/a[not(contains(@class,"ico-tv-tournament"))]').text
                            odd1=match.find_element_by_xpath('./td[3]/a').text
                            odd2=match.find_element_by_xpath('./td[4]/a').text
                            winnerP1=np.nan
                        
                        players=players.replace(". ",".")
                        player1=players.split("-")[0]
                        player2=players.split("- ")[1]
                        surface = re.search('\(([^)]+)', h1).group(1)
                        list_location.append(location)
                        list_surface.append(surface)
                        list_player1.append(player1)
                        list_player2.append(player2)
                        list_odd1.append(odd1)
                        list_odd2.append(odd2)
                        list_winnerP1.append(winnerP1)
                    except :
                        continue
                    
        except NoSuchElementException:
            continue
    driver.quit()
dictionnary={"Location":list_location,"Surface":list_surface,"Player1":list_player1,"Player2":list_player2,"AvgP1":list_odd1,"AvgP2":list_odd2,"WinnerP1":list_winnerP1}

            
    

In [3]:
print(dictionnary)

{'Location': ['Cagliari', 'Cagliari', 'Cagliari', 'Marbella', 'Marbella', 'Marbella', 'Marbella'], 'Surface': ['clay', 'clay', 'clay', 'clay', 'clay', 'clay', 'clay'], 'Player1': ['Gerasimov E.', 'Basilashvili N.', 'Martin A.', 'Gombos N.', 'Bagnis F.', 'Ramos A.', 'Carreno'], 'Player2': ['Bedene A.', 'Kovalik J.', 'Fritz T.', 'Delbonis F.', 'Kwon Soonwoo', 'Berankis R.', 'Vilella M.M.'], 'AvgP1': ['1.96', '1.79', '2.42', '3.04', '1.31', '1.24', '1.41'], 'AvgP2': ['1.86', '2.04', '1.57', '1.39', '3.44', '4.05', '2.92'], 'WinnerP1': [nan, nan, nan, nan, nan, nan, nan]}


In [6]:
df_validation=pd.DataFrame(dictionnary)

In [25]:
locations_list=df_validation['Location'].unique()

In [29]:
print(type(locations_list))

<class 'numpy.ndarray'>


In [41]:
url_tourn="https://www.atptour.com/en/tournaments/"

driver = webdriver.Chrome(executable_path=r'/Users/pierremecchia/Desktop/chromedriver')
driver.get(url_tourn)
sleep(5)

month_tournament=driver.find_elements_by_xpath("//*[@id='contentAccordionWrapper']/div[contains(@class,'expand')][1]/div[2]/div/table/tbody/tr")

list_series=[]
list_court=[]
list_location=[]

for tournament in month_tournament:
    #print(tournament)
    try:
        location=tournament.find_element_by_xpath('./td[2]/span[1]').text
        town=location.split(",")[0]
        if town in locations_list:
            try:
                series=tournament.find_element_by_xpath('./td[1]/img').get_attribute("src")
                if "250.png" in series:
                    series="ATP250"
                elif "500.png" in series:
                    series="ATP500"
                elif "1000.png" in series:
                    series="Masters 1000"
                elif "grandslam.png" in series:
                    series="Grand Slam"
                elif "finals.svg" in series:
                    series="ATP Finals"
                else:
                    series=np.nan
            except:
                series=np.nan
            try:
                court=tournament.find_element_by_xpath('./td[3]/table/tbody/tr/td[2]/div/div').text
                court=court.split(" ")[0]
            except:
                court=np.nan
            list_series.append(series)
            list_court.append(court)
            list_location.append(town)
    except:
        print("nan")

driver.quit()
loc_dict={"Location":list_location,"Series":list_series,"Court":list_court}

In [42]:
print(loc_dict)

{'Location': ['Marbella', 'Cagliari'], 'Series': ['ATP250', 'ATP250'], 'Court': ['Outdoor', 'Outdoor']}
